## Example 1: Aggregation: Simple Grouping

```javascript
// Group by author and count posts
db.posts.aggregate([
  { $group: { _id: "$author", totalPosts: { $sum: 1 } } }
])
```
**Explanation:** The aggregation pipeline allows data processing. Here, we group by the author field and count the number of posts per author.


## Example 2: Aggregation: Sorting and Limiting

```javascript
// Sort by date and get the latest 5 posts
db.posts.aggregate([
  { $sort: { date: -1 } },
  { $limit: 5 }
])
```
**Explanation:** We sort the documents by date in descending order and then limit to 5.


## Example 3: Aggregation: Unwind Arrays

```javascript
// Unwind the tags array and group by tag
db.posts.aggregate([
  { $unwind: "$tags" },
  { $group: { _id: "$tags", count: { $sum: 1 } } }
])
```
**Explanation:** $unwind deconstructs an array field, creating a document for each element. Then we group by the tag and count.


## Example 4: Aggregation: Lookup (Join)

```javascript
// Suppose we have a separate authors collection
db.authors.insertMany([
  { _id: 1, name: "John Doe", email: "john@example.com" },
  { _id: 2, name: "Jane Smith", email: "jane@example.com" }
])
```

// Now, let's change the posts collection to have an authorId instead of author name
db.posts.updateMany({}, [
  { $set: { authorId: 1 } } // This is just an example; in reality, we'd update appropriately.
])

// Then, we can use lookup to join
db.posts.aggregate([
  {
    $lookup: {
      from: "authors",
      localField: "authorId",
      foreignField: "_id",
      as: "authorInfo"
    }
  }
])
Explanation: $lookup performs a left outer join to another collection (from) and adds an array of matching documents to the input document.

Example 13: Aggregation: Project and Shape Data
javascript
// Project only the title and author's name
db.posts.aggregate([
  {
    $lookup: {
      from: "authors",
      localField: "authorId",
      foreignField: "_id",
      as: "authorInfo"
    }
  },
  {
    $project: {
      title: 1,
      authorName: { $arrayElemAt: ["$authorInfo.name", 0] }
    }
  }
])
Explanation: $project reshapes the document. We use $arrayElemAt to get the first element of the authorInfo array (since we expect one author per post).

Example 14: Aggregation: Conditional Logic
javascript
// Categorize posts by length of content
db.posts.aggregate([
  {
    $project: {
      title: 1,
      contentLength: { $strLenCP: "$content" },
      category: {
        $cond: {
          if: { $gt: [{ $strLenCP: "$content" }, 100] },
          then: "Long",
          else: "Short"
        }
      }
    }
  }
])
Explanation: We use $cond to conditionally assign a category based on the length of the content.

Example 15: Aggregation: Facet for Multiple Aggregations
javascript
// Get multiple aggregations in one query
db.posts.aggregate([
  {
    $facet: {
      "byAuthor": [
        { $group: { _id: "$author", count: { $sum: 1 } } }
      ],
      "byTag": [
        { $unwind: "$tags" },
        { $group: { _id: "$tags", count: { $sum: 1 } } }
      ],
      "totalViews": [
        { $group: { _id: null, total: { $sum: "$views" } } }
      ]
    }
  }
])
Explanation: $facet allows multiple aggregation pipelines to be executed on the same set of input documents.

Example 16: Transactions (ACID)
javascript
// Start a session
const session = db.getMongo().startSession();

// Start a transaction
session.startTransaction();

try {
  // Operations within the transaction
  db.posts.insertOne({ title: "New Post" }, { session });
  db.authors.updateOne({ _id: 1 }, { $inc: { postCount: 1 } }, { session });

  // Commit the transaction
  session.commitTransaction();
} catch (error) {
  // Abort transaction on error
  session.abortTransaction();
} finally {
  session.endSession();
}